# **Pandas y sqlite3**

## Objetivos

Después de completar notebook será capas de:

*   Crear una base de datos sqlite3.
*   Consultar una base de datos sqlite3 y convertirla en un *dataframe*.

La función `read_sql()` de **Pandas** es usada para leer consultas SQL o tablas de bases de datos a un *dataframe*. Esta función también contiene a las funciones `read_sql_query()` y `read_sql_table()`. Estas funciones retornan una tabla SQL como una estructura de datos de dos dimensiones con sus ejes etiquetados.
En este laboratorio se explicará el uso de la función de **Pandas** `read_sql()` a través de una serie de ejemplos.

## Tabla de contenidos
1. [Sintaxis de read_sql()](#0)
2. [Crear base de datos y tabla](#2)
3. [Insertar datos en la tabla](#4)
4. [Ejecutar consulta SQL usando read_sql()](#6)
5. [Uso de read_sql_query()](#8)
6. [Uso de read_sql_table()](#10)
7. [Filtrar filas de una tabla SQL](#12)
8. [Uso del método fetchall() de un Cursor](#14)


# 1. Sintaxis de read_sql() <a id="0"></a>

A continuación se muestra la sintaxis completa de las funciones `read_sql()`, `read_sql_query()` y `read_sql_table()`. Todas estas funciones retornan un *dataframe*.

```python
# Sintaxis de read_sql()
pandas.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)

# Sintaxis de read_sql_query()
pandas.read_sql_query(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None, dtype=None)

# Sintaxis de read_sql_table()
pandas.read_sql_table(table_name, con, schema=None, index_col=None, coerce_float=True, parse_dates=None, columns=None, chunksize=None)
```

Más información acerca de los parámetros de estas funciones se puede encontrar en:

- https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html
- https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
- https://pandas.pydata.org/docs/reference/api/pandas.read_sql_table.html

Antes de empezar a aprender el uso de la función de **Pandas** `read_sql()` y derivados, se procederá a crear una base de datos y una tabla utilizando la base de datos liviana **sqlite3**.

# 2. Crear base de datos y tabla  <a id="2"></a>

El siguiente ejemplo se puede utilizar para crear una base de datos y una tabla en *Python* utilizando la biblioteca `sqlite3`. Si no estuviera esta biblioteca disponible, se puede instalar utilizado el comando `pip`. Para poder utilizar esta biblioteca primero hay que preceder a su importación.

In [ ]:
import sqlite3
import pandas as pd

#Conectar a la Base de Datos
conexion = sqlite3.connect('datos/base_datos_cursos.db') 
cur = conexion.cursor()

# Crea una Tabla
cur.execute('''CREATE TABLE IF NOT EXISTS cursos
          ([curso_id] INTEGER PRIMARY KEY, 
           [curso_nombre] TEXT, 
           [valor] INTEGER, 
           [duracion] TEXT, 
           [descuento] INTEGER)''')

Al ejecutar las instrucciones previas se creó una nueva base de datos llamada `base_datos_cursos.db` con una tabla con el nombre de `cursos`.

# 3. Insertar datos en la tabla  <a id="4"></a>

Ahora se va a proceder a insertar algunas filas o registros en la tabla utilizando la función `execute()` del objeto `Cursor`. No hay que olvidar al final ejecutar la confirmación de las acciones realizadas en la base da datos a través de la función `commit()` para que las filas insertadas en la base de datos sean guardadas de forma permanente.

In [ ]:
# Inserta algunos registros o filas
cur.execute('''INSERT INTO cursos (curso_id, curso_nombre, valor, duracion, descuento)
                VALUES (1,'Numpy',250000,'50 Días', 20000),
                (2,'Pandas',200000,'35 Días', 10000),
                (3,'Java',150000,'35 Días', 8000),
                (4,'Python',150000,'30 Días', 5000),
                (5,'PHP',280000,'30 Días', 8000)''') 
                    
# Confirmar la Inserción
conexion.commit()

# 4. Ejecutar consulta SQL usando read_sql()  <a id="6"></a>

Ya creada la base de datos con una tabla, se puede utilizar la función `read_sql()` para cargar la tabla. Esto se puede realizar, ya sea, pasando como parámetro una consulta SQL o por el nombre de la tabla. Para el caso que se va a presentar a continuación, como el primer parámetro es una consulta SQL, internamente la función `read_sql()` llama a la función `sql_read_query()`.

In [ ]:
# Ejecutar consulta SQL          
consulta_sql = pd.read_sql('SELECT * FROM cursos', conexion)

# Convierte de SQL a dataFrame
df = pd.DataFrame(consulta_sql, columns = ['curso_id', 'curso_nombre', 'valor','duracion','descuento'])
df

# 5. Uso de read_sql_query()  <a id="8"></a>

De igual forma, se puede reescribir las instrucciones previas utilizando de forma directa la función `read_sql_query()`.

In [ ]:
# Ejecutar consulta SQL          
consulta_sql = pd.read_sql_query('SELECT * FROM cursos', conexion)

# Convierte de SQL a dataFrame
df = pd.DataFrame(consulta_sql, columns = ['curso_id', 'curso_nombre', 'valor','duracion','descuento'])
df

# 6. Uso de read_sql_table()  <a id="10"></a>

En los ejemplos anteriores, solo se ha estado usando consultas SQL para leer la tabla y convertirla en un *dataframe* de **Pandas**. Ahora, se va a utilizar el nombre de la tabla para cargar la tabla completa con ayuda de la función `read_sql_table()`. Esta función carga todas las filas en el *dataframe*.

In [ ]:
import sqlalchemy

conexion2 = sqlalchemy.create_engine('sqlite:///datos/base_datos_cursos.db')

# Ejecutar consulta SQL          
consulta_sql = pd.read_sql_table('cursos', conexion2)

# Convierte de SQL a dataFrame
df = pd.DataFrame(consulta_sql, columns = ['curso_id', 'curso_nombre', 'valor','duracion','descuento'])
df

# 7. Filtrar filas de una tabla SQL  <a id="12"></a>

En muchas ocasiones no es necesario leer todas las filas o registros desde una tabla SQL. Para cargar solo un grupo seleccionado de filas basado en una condición, se puede usar la cláusula `WHERE` de **SQL**.

In [ ]:
# Ejecutar consulta SQL          
consulta_sql = pd.read_sql_query('SELECT * FROM cursos WHERE valor = 150000', conexion)

# Convierte de SQL a dataFrame
df = pd.DataFrame(consulta_sql, columns = ['curso_id', 'curso_nombre', 'valor','duracion','descuento'])
print(df)

# 8. Uso del método fetchall() de un Cursor  <a id="14"></a>

De forma alternativa, se puede hacer uso del constructor del *dataframe* a través del método `Cursor.fetchall()` para cargar la tabla SQL en el *dataframe*. Para obtener el mismo resultado del ejemplo anterior se puede realizar lo siguiente.

In [ ]:
# Ejecutar consulta SQL  
         
cur.execute('SELECT * FROM cursos WHERE valor = 150000')

# Convierte de SQL a dataFrame
df = pd.DataFrame(cur.fetchall(), columns = ['curso_id', 'curso_nombre', 'valor','duracion','descuento'])
print(df)


Finalmente, cuando ya no se va a hacer más uso de la base de datos, esta se debe cerrar con la función `close()`.

In [ ]:
# cerrar la conexión a la base de datos

conexion.close()
conexion2.dispose()

Para más información de como trabajar con *Python* y bases de datos *SQLite* se puede visitar los siguientes links:
- https://www.sqlite.org/index.html
- https://docs.python.org/es/3/library/sqlite3.html